### Given the data points (observed decisions), inferred constraints and the goodness-of-fit metric using exact or greedy heuristics are inferred.

In [2]:
import pandas as pd
import numpy as np
from scipy.spatial import ConvexHull
from numpy import linalg as LA
import itertools
import sys
import time

In [3]:
data = pd.read_csv(r'Data/1Final_530_Female_1959_server.csv')
# data = data.drop(data.index[30:])

X = data

In [4]:
class IO:
    def __init__(self, X, num_Unknown_cons):
        self.num_points = X.shape[0]
        self.num_vars = X.shape[1]
        self.num_Unknown_cons = num_Unknown_cons
        self.X = X

    def convexhull(self):
        points = self.X.to_numpy() # CONVERTING THE DATAFRAME TO ARRAY
        hull = ConvexHull(points)  # FINDING THE CONVEX HULL OF DATA POINTS
        eq = hull.equations   # GETTING THE EQUATIONS FORMING THE CONVEX HULL
        hul = pd.DataFrame(eq, columns= self.X.columns.values.tolist() + ["RHS"])  # CREATING A DATAFRAME WITH THE COEFFICIENTS OF CONSTRAINTS
        num_hul_eq = hul.shape[0]   # GETTING THE NUMBER OF EQUATIONS FORMING THE CONVEX HULL
        hul.iloc[0:num_hul_eq, 0:self.num_vars] *= -1  # MAKING AX = b INSTEAD OF AX + b = 0
        hul.iloc[0:num_hul_eq, self.num_vars] = np.round(hul.iloc[0:num_hul_eq, self.num_vars], decimals=5)  # ROUNDING TO 5 DECIMALS
        output = {"hul":hul, "num_hul_eq": num_hul_eq}
        return output
    
    def sumdistance(self, output):
        gre_distance = np.zeros([self.num_points,output["num_hul_eq"]])
        for k in range(self.num_points):
            p = self.X.iloc[k].to_numpy()   # p: EACH POINT
            for i in range(output["num_hul_eq"]):  # i: EACH EQUATION (POTENTIAL INFERRED CONSTRAINT)
                a = output["hul"].iloc[i,:self.num_vars].to_numpy()
                b = output["hul"].iloc[i,self.num_vars]
                gre_distance[k,i] = abs(np.dot(p,a) - b)   /  LA.norm( a ) # EUCLIDEAN DISTANCE OF POINT p FROM HYPERPLANE i
        gre_distance = np.round(gre_distance, decimals=10) # ROUNDING TO 10 DECIMALS
        sum_each_i = gre_distance.sum(axis=0)  # FOR EACH i, GETTING THE SUM DISTANCE OF ALL POINTS FROM IT
        return sum_each_i

    def gre_infer(self, sum_each_i,output):
        sorted_indices = np.argsort(sum_each_i)  # SORTING THE EQUATIONS (POTENTIAL INFERRED CONSTRAINT) BASED ON THE SUM DISTANCES
        smallest_indices = sorted_indices[:self.num_Unknown_cons]  # CHOOSING THE INDECIS OF num_Unknown_cons (exp. 2) HYPERPLANES WITH THE MINIMUM SUM DISTANCES
        inferred_cons = [output["hul"].iloc[ind].to_numpy() for ind in smallest_indices] # FINDING THE INFERRED HYPERPLANES BASED ON THEIR INDECIS
        return inferred_cons


    def combinations(self, output):
        index_of_hul = [i for i in range(output["num_hul_eq"])] # 
        comb = list(itertools.combinations(index_of_hul, self.num_Unknown_cons)) # FINDING POSSIBLE COMBINATIONS OF num_Unknown_cons CONSTRAINTS OUT OF TOTAL NUMBER OF EQUATIONS FORMING THE CONVEX HULL (index_of_hul)
        return comb
    
    def IO_metrics(self, output, comb):
        metric_comb = [] #len(comb)
        for c in comb: # c: EACH COMBINATION (POTENTIAL SET OF INFERRED CONSTRAINT(S))
            distance = np.zeros([self.num_points,self.num_Unknown_cons])
            co = 0  # co : COUNTER FOR THE EQUATION IN EACH COMBINATION
            for i in c: # i: INDEX OF EACH EQUATION IN COMBINATION c
                a = output["hul"].iloc[i,:self.num_vars].to_numpy()
                b = output["hul"].iloc[i,self.num_vars]
                for k in range(self.num_points):
                    p = self.X.iloc[k].to_numpy()  # p: EACH POINT
                    distance[k,co] = abs(np.dot(p,a) - b)   /  LA.norm( a ) # EUCLIDEAN DISTANCE OF POINT p FROM HYPERPLANE i IN COMBINATION c
                co +=1
            min_distance = np.amin(distance, axis=1)  # FINDING THE MIN DISTANCE OF POINT p FROM HYPERPLANES IN COMBINATION c
            metric_comb.append((min_distance.sum()) / self.num_points ) # FINDING THE VALUE OF MERIC FOR COMBINATION c
        return metric_comb
          
    def infer(self, output, comb, metric_comb):      
        sorted_indices = np.argsort(metric_comb) # SORTING THE COMBINATION (POTENTIAL SET OF INFERRED CONSTRAINT(S)) BASED ON THE VALUE OF METRIC
        smallest_indices = sorted_indices[:1]  # CHOOSING THE COMBINATION WITH THE MINIMUM VALUE OF METRIC
        inferred_cons = [output["hul"].iloc[ind].to_numpy() for ind in comb[smallest_indices[0]]]  # FINDING THE INFERRED HYPERPLANES BASED ON THE INDEX OF THE OPTIMAL COMBIMATION
        metric = min(metric_comb) # GETTING THE MINIMUM VALUE OF METRIC
        final = {"metric":metric, "inferred_cons": inferred_cons}
        return final

#### Run the following cell to solve the model using the EXACT heuristic method.

In [ ]:
# # Use the following four lines if arguments were passed to the script as the number of inferred constraints. 
# if len(sys.argv)>1:  
#     num_Unknown_cons = int(sys.argv[1]) 
# else:
#     num_Unknown_cons = 1


num_Unknown_cons = 2 # CHOOSE THE NUMBER OF INFERRED CONSTRAINTS    
start_time = time.time()
ob = IO(X,num_Unknown_cons)  # CREATE AN OBJECT FROM CLASS IO
output = ob.convexhull()
output = ob.convexhull()
comb = ob.combinations(output)
metric_comb = ob.IO_metrics(output, comb)
final = ob.infer(output, comb, metric_comb)
metric = final["metric"]
inferred_cons = final["inferred_cons"]

print("--- %s seconds ---" % (time.time() - start_time))

# OUTPUT
print("num of hull eq: ", output['num_hul_eq'])
print("num of combinations: ", len(comb))
print("(j,k,i): ", X.shape[1],X.shape[0], num_Unknown_cons)
print("metric: ", metric)
print("constraints: ", inferred_cons)

#### Run the following cell to solve the model using the GREEDY heuristic method.

In [ ]:
# # Use the following four lines if arguments were passed to the script as the number of inferred constraints. 
# if len(sys.argv)>1:  
#     num_Unknown_cons = int(sys.argv[1]) 
# else:
#     num_Unknown_cons = 1


num_Unknown_cons = 2 # CHOOSE THE NUMBER OF INFERRED CONSTRAINTS   start_time = time.time()
ob = IO(X,num_Unknown_cons)
output = ob.convexhull()
sum_each_i = ob.sumdistance(output)
inferred_cons = ob.gre_infer(sum_each_i,output)
out = pd.DataFrame(inferred_cons, columns = ['Fruits', 'Vegetables', 'Grains','Protein','Dairy','RHS'])
metric= ob.gre_IO_metrics(out)

print("--- %s seconds ---" % (time.time() - start_time))

# OUTPUT
print("num of hull eq: ", output['num_hul_eq'])
print("(j,k,i): ", X.shape[1],X.shape[0], num_Unknown_cons)
print("metric: ", metric)
print("constraints: ", inferred_cons)